In [23]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
plotly.offline.init_notebook_mode (connected = True)
import matplotlib.pyplot as plt
import warnings

In [24]:
header = 1 # 0 for No, 1 for Yes
label = 1 # 0 for No, 1 for Yes
path="test-data-revised.csv" #path to the datset
visualization=1 # 0 for No, 1 for Yes
apply = 1 # 0 for none, 1 for standardization, 2 for normalization
components = 2 # get number input from user

In [25]:
if header==1:
    df = pd.read_csv(path)
else:
    df = pd.read_csv(path,header=None)

if label==1:
    labeled=df[df.columns[-1]]
    df=df.drop([df.columns[-1]],axis=1)

In [26]:
warnings.filterwarnings("ignore")

In [27]:
feats = df.columns[:]
feats # getting the header names

Index(['A', 'B', 'C'], dtype='object')

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

if apply==1:
    scaler = StandardScaler().fit_transform(df[feats])
elif apply==2:
    scaler = preprocessing.normalize(df[feats])
else:
    scaler = df[feats]

In [29]:
scaled = pd.DataFrame(scaler, columns=feats)
if label==1:
    scaled['Label'] = labeled

In [30]:
#before applying PCA Visualization #Shivesh add this as title
if label==1 and visualization==1 and len(feats)>components:
    features = feats
    fig = px.scatter_matrix(
        scaled,
        dimensions=features,
        color="Label"
    )
    fig.update_traces(diagonal_visible=True)
    fig.show()

In [31]:
from sklearn.decomposition import PCA
def create_pca_label(X, y, n_components=None):
    if (n_components != None):
        pca = PCA(n_components)
    else:
        pca = PCA()
    trans = pca.fit_transform(X)
    trans = pd.DataFrame(trans)
    trans['species'] = y
    return pca, trans

def create_pca(X, n_components=None):
    if (n_components != None):
        pca = PCA(n_components)
    else:
        pca = PCA()
    trans = pca.fit_transform(X)
    trans = pd.DataFrame(trans)
    return pca, trans

In [32]:
if label==1:
    X, y = scaled.drop(['Label'], axis=1), scaled['Label']

    if len(feats)>components:
        if components == 0:
            components=None
        pca1, trans1 = create_pca_label(X=X, y=y, n_components=components)
        print(pca1.explained_variance_ratio_)#mention what it is : Shivesh
        print(pca1.components_)# mention this to!
    else:
        print("No of Components entered is greater than number of input features.")
else:
    X = scaled

    if len(feats)>components:
        if components == 0:
            components=None
        pca1, trans1 = create_pca(X=X, n_components=components)
        print(pca1.explained_variance_ratio_)#mention what it is : Shivesh
        print(pca1.components_)# mention this to!

    

[0.7568932 0.2431068]
[[-0.64202376  0.64202376 -0.41905964]
 [ 0.29631992 -0.29631992 -0.90795871]]


In [33]:
if label==1 and visualization==1 and len(feats)>components:

    pca = PCA()
    components = pca.fit_transform(df[features])
    labels = {
        str(i): f"PC {i+1} ({var:.1f}%)"
        for i, var in enumerate(pca.explained_variance_ratio_ * 100)
    }
    fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(len(features)),
    color=scaled["Label"]
    )
    fig.update_traces(diagonal_visible=True)
    fig.show()